In [5]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from db import helper as db_helper
from db.models import SpotRate
import pandas as pd

In [ ]:
# spot_data = db_helper.pull_scraped_table()

In [ ]:
# for row in spot_data:
#     print(dir(row))
#     print(row.url)
#     break


# # df = pd.DataFrame(spot_data)
# # df

In [11]:
session = db_helper.start_session()

df = pd.read_sql_table('spot_rates', session.bind)

2018-12-27 12:35:25,153 INFO sqlalchemy.engine.base.Engine select version()
2018-12-27 12:35:25,155 INFO sqlalchemy.engine.base.Engine {}
2018-12-27 12:35:25,157 INFO sqlalchemy.engine.base.Engine select current_schema()
2018-12-27 12:35:25,158 INFO sqlalchemy.engine.base.Engine {}
2018-12-27 12:35:25,163 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-12-27 12:35:25,166 INFO sqlalchemy.engine.base.Engine {}
2018-12-27 12:35:25,171 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-12-27 12:35:25,174 INFO sqlalchemy.engine.base.Engine {}
2018-12-27 12:35:25,179 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2018-12-27 12:35:25,181 INFO sqlalchemy.engine.base.Engine {}
2018-12-27 12:35:25,184 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind = 'r'
2018-12-27 12:

2018-12-27 12:35:25,294 INFO sqlalchemy.engine.base.Engine {}


In [25]:
# Quick snapshot of the data
df.describe()

# lack of significant data in the 52wk columns
df.drop(columns=['id', 'url', 'target_52wk_high', 'target_52wk_low', 'created_at', 'updated_at'], inplace=True)

KeyError: "['id' 'url' 'target_52wk_high' 'target_52wk_low' 'created_at' 'updated_at'] not found in axis"

In [26]:
# create Date
import datetime
df['date'] = df.apply(lambda x: datetime.date(x['year'], x['month'], x['day']), axis=1)
df.drop(columns=['day', 'month', 'year'], inplace=True)

df

,base_currency,target_currency,base_value,target_spot_rate,date
0,GBP,Japanese Yen,1.0,130.5530,2016-08-12
1,GBP,Malaysian ringgit,1.0,5.2109,2016-08-12
2,GBP,New Zealand Dollar,1.0,1.7886,2016-08-12
3,GBP,Norwegian Krone,1.0,10.6189,2016-08-12
4,GBP,Polish Zloty,1.0,4.9372,2016-08-12
5,GBP,Russian Ruble,1.0,83.8059,2016-08-12
6,GBP,Saudi Riyal,1.0,4.8510,2016-08-12
7,GBP,Singapore Dollar,1.0,1.7391,2016-08-12
8,GBP,South African Rand,1.0,17.3885,2016-08-12
9,GBP,South Korean Won,1.0,1424.7579,2016-08-12
